# Introduction

In this tutorial, we will take a closer look at strategies in the context of both multi-step forecasting and simultaneous forecasting of multiple time series.

We will explore how strategies affect the feature generation process used for model training.

Let's import everything we need and define functions to obtain the results.

In [11]:
import warnings

warnings.filterwarnings("ignore")

from copy import deepcopy
from typing import List, Optional, Union

import numpy as np
import pandas as pd

from tsururu.dataset import IndexSlicer, Pipeline, TSDataset
from tsururu.model_training.trainer import DLTrainer, MLTrainer
from tsururu.model_training.validator import KFoldCrossValidator
from tsururu.models.boost import CatBoost
from tsururu.strategies import (
    DirectStrategy,
    FlatWideMIMOStrategy,
    MIMOStrategy,
    RecursiveStrategy,
)
from tsururu.strategies.base import Strategy
from tsururu.strategies.utils import timing_decorator
from tsururu.transformers import (
    DateSeasonsGenerator,
    DifferenceNormalizer,
    LagTransformer,
    LastKnownNormalizer,
    SequentialTransformer,
    StandardScalerTransformer,
    TargetGenerator,
    UnionTransformer,
)

index_slicer = IndexSlicer()

Let's transfer the necessary code into the notebook so that we can output the values of variables.

In [12]:
class RecursiveStrategy(Strategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
        model_horizon: int = 1,
        reduced: bool = False,
    ):
        super().__init__(horizon, history, trainer, pipeline, step)
        self.model_horizon = model_horizon
        self.reduced = reduced
        self.strategy_name = "recursive"

    @timing_decorator
    def fit(
        self,
        dataset: TSDataset,
    ) -> "RecursiveStrategy":
        features_idx = index_slicer.create_idx_data(
            dataset.data,
            self.model_horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )

        target_idx = index_slicer.create_idx_target(
            dataset.data,
            self.model_horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )

        data = self.pipeline.create_data_dict_for_pipeline(dataset, features_idx, target_idx)
        data = self.pipeline.fit_transform(data, self.strategy_name)

        val_dataset = self.trainer.validation_params.get("validation_data")

        if val_dataset:
            val_features_idx = index_slicer.create_idx_data(
                val_dataset.data,
                self.model_horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            val_target_idx = index_slicer.create_idx_target(
                val_dataset.data,
                self.model_horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            val_data = self.pipeline.create_data_dict_for_pipeline(
                val_dataset, val_features_idx, val_target_idx
            )
            val_data = self.pipeline.transform(val_data)
        else:
            val_data = None

        if isinstance(self.trainer, DLTrainer):
            if self.strategy_name == "FlatWideMIMOStrategy":
                self.trainer.horizon = 1
            else:
                self.trainer.horizon = self.model_horizon
            self.trainer.history = self.history

        current_trainer = deepcopy(self.trainer)

        # In Recursive strategy, we train the individual model
        if isinstance(current_trainer, DLTrainer):
            checkpoint_path = current_trainer.checkpoint_path
            pretrained_path = current_trainer.pretrained_path

            current_trainer.checkpoint_path /= "trainer_0"
            if pretrained_path:
                current_trainer.pretrained_path /= "trainer_0"

        current_trainer.fit(data, self.pipeline, val_data)

        if isinstance(current_trainer, DLTrainer):
            current_trainer.checkpoint_path = checkpoint_path
            current_trainer.pretrained_path = pretrained_path

        self.trainers.append(current_trainer)
        return self

    def make_step(self, step: int, dataset: TSDataset) -> TSDataset:
        test_idx = index_slicer.create_idx_test(
            dataset.data,
            self.horizon - step * self.model_horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )

        target_idx = index_slicer.create_idx_target(
            dataset.data,
            self.horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )[:, self.model_horizon * step : self.model_horizon * (step + 1)]

        data = self.pipeline.create_data_dict_for_pipeline(dataset, test_idx, target_idx)
        data = self.pipeline.transform(data)

        pred = self.trainers[0].predict(data, self.pipeline)
        pred = self.pipeline.inverse_transform_y(pred)

        dataset.data.loc[target_idx.reshape(-1), dataset.target_column] = pred.reshape(-1)

        return dataset

    @timing_decorator
    def predict(self, dataset: TSDataset, test_all: bool = False) -> pd.DataFrame:
        new_data = dataset.make_padded_test(
            self.horizon, self.history, test_all=test_all, step=self.step
        )
        new_dataset = TSDataset(new_data, dataset.columns_params, dataset.delta)

        if test_all:
            new_dataset.data = new_dataset.data.sort_values(
                [dataset.id_column, "segment_col", dataset.date_column]
            )

        if self.reduced:
            current_test_ids = index_slicer.create_idx_data(
                new_dataset.data,
                self.model_horizon,
                self.history,
                step=self.model_horizon,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            target_ids = index_slicer.create_idx_target(
                new_dataset.data,
                self.horizon,
                self.history,
                step=self.model_horizon,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            data = self.pipeline.create_data_dict_for_pipeline(
                new_dataset, current_test_ids, target_ids
            )
            data = self.pipeline.transform(data)

            pred = self.trainers[0].predict(data, self.pipeline)
            pred = self.pipeline.inverse_transform_y(pred)

            new_dataset.data.loc[target_ids.reshape(-1), dataset.target_column] = pred.reshape(-1)

        else:
            for step in range(self.horizon // self.model_horizon):
                new_dataset = self.make_step(step, new_dataset)

        # Get dataframe with predictions only
        pred_df = self._make_preds_df(new_dataset, self.horizon, self.history)
        return pred_df

class MIMOStrategy(RecursiveStrategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
    ):
        super().__init__(horizon, history, trainer, pipeline, step, model_horizon=horizon)
        self.strategy_name = "MIMOStrategy"

class DirectStrategy(RecursiveStrategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
        model_horizon: int = 1,
        equal_train_size: bool = False,
    ):
        super().__init__(horizon, history, trainer, pipeline, step, model_horizon)
        self.equal_train_size = equal_train_size
        self.strategy_name = "direct"

    @timing_decorator
    def fit(
        self,
        dataset: TSDataset,
    ) -> "DirectStrategy":
        self.trainers = []

        if self.equal_train_size:
            features_idx = index_slicer.create_idx_data(
                dataset.data,
                self.model_horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            target_idx = index_slicer.create_idx_target(
                dataset.data,
                self.model_horizon,
                self.history,
                self.step,
                date_column=dataset.date_column,
                delta=dataset.delta,
            )

            data = self.pipeline.create_data_dict_for_pipeline(dataset, features_idx, target_idx)
            data = self.pipeline.fit_transform(data, self.strategy_name)

            val_dataset = self.trainer.validation_params.get("validation_data")

            if val_dataset:
                val_features_idx = index_slicer.create_idx_data(
                    val_dataset.data,
                    self.model_horizon,
                    self.history,
                    self.step,
                    date_column=val_dataset.date_column,
                    delta=val_dataset.delta,
                )

                val_target_idx = index_slicer.create_idx_target(
                    val_dataset.data,
                    self.model_horizon,
                    self.history,
                    self.step,
                    date_column=val_dataset.date_column,
                    delta=val_dataset.delta,
                )

                val_data = self.pipeline.create_data_dict_for_pipeline(
                    val_dataset, val_features_idx, val_target_idx
                )
                val_data = self.pipeline.transform(val_data)
            else:
                val_data = None

            for model_i, horizon in enumerate(range(1, self.horizon // self.model_horizon + 1)):
                target_idx = index_slicer.create_idx_target(
                    dataset.data,
                    self.horizon,
                    self.history,
                    self.step,
                    date_column=dataset.date_column,
                    delta=dataset.delta,
                )[:, (horizon - 1) * self.model_horizon : horizon * self.model_horizon]

                data["target_idx"] = target_idx

                if val_dataset:
                    val_target_idx = index_slicer.create_idx_target(
                        val_dataset.data,
                        self.horizon,
                        self.history,
                        self.step,
                        date_column=val_dataset.date_column,
                        delta=val_dataset.delta,
                    )[:, (horizon - 1) * self.model_horizon : horizon * self.model_horizon]

                    val_data["target_idx"] = val_target_idx

                if isinstance(self.trainer, DLTrainer):
                    self.trainer.horizon = self.model_horizon
                    self.trainer.history = self.history

                current_trainer = deepcopy(self.trainer)

                # In Direct strategy, we train several models, one for each model_horizon
                if isinstance(current_trainer, DLTrainer):
                    checkpoint_path = current_trainer.checkpoint_path
                    pretrained_path = current_trainer.pretrained_path

                    current_trainer.checkpoint_path /= f"trainer_{model_i}"
                    if pretrained_path:
                        current_trainer.pretrained_path /= f"trainer_{model_i}"

                current_trainer.fit(data, self.pipeline, val_data)

                if isinstance(current_trainer, DLTrainer):
                    current_trainer.checkpoint_path = checkpoint_path
                    current_trainer.pretrained_path = pretrained_path

                self.trainers.append(current_trainer)

        else:
            for model_i, horizon in enumerate(range(1, self.horizon // self.model_horizon + 1)):
                features_idx = index_slicer.create_idx_data(
                    dataset.data,
                    self.model_horizon * horizon,
                    self.history,
                    self.step,
                    date_column=dataset.date_column,
                    delta=dataset.delta,
                )

                target_idx = index_slicer.create_idx_target(
                    dataset.data,
                    self.model_horizon * horizon,
                    self.history,
                    self.step,
                    date_column=dataset.date_column,
                    delta=dataset.delta,
                    n_last_horizon=self.model_horizon,
                )

                data = self.pipeline.create_data_dict_for_pipeline(
                    dataset, features_idx, target_idx
                )
                data = self.pipeline.fit_transform(data, self.strategy_name)

                val_dataset = self.trainer.validation_params.get("validation_data")

                if val_dataset:
                    val_features_idx = index_slicer.create_idx_data(
                        val_dataset.data,
                        self.model_horizon * horizon,
                        self.history,
                        self.step,
                        date_column=val_dataset.date_column,
                        delta=val_dataset.delta,
                    )

                    val_target_idx = index_slicer.create_idx_target(
                        val_dataset.data,
                        self.model_horizon * horizon,
                        self.history,
                        self.step,
                        date_column=val_dataset.date_column,
                        delta=val_dataset.delta,
                        n_last_horizon=self.model_horizon,
                    )

                    val_data = self.pipeline.create_data_dict_for_pipeline(
                        val_dataset, val_features_idx, val_target_idx
                    )
                    val_data = self.pipeline.transform(val_data)
                else:
                    val_data = None

                if isinstance(self.trainer, DLTrainer):
                    self.trainer.horizon = self.model_horizon
                    self.trainer.history = self.history

                current_trainer = deepcopy(self.trainer)

                # In Direct strategy, we train several models, one for each model_horizon
                if isinstance(current_trainer, DLTrainer):
                    checkpoint_path = current_trainer.checkpoint_path
                    pretrained_path = current_trainer.pretrained_path

                    current_trainer.checkpoint_path /= f"trainer_{model_i}"
                    if pretrained_path:
                        current_trainer.pretrained_path /= f"trainer_{model_i}"

                current_trainer.fit(data, self.pipeline, val_data)

                if isinstance(current_trainer, DLTrainer):
                    current_trainer.checkpoint_path = checkpoint_path
                    current_trainer.pretrained_path = pretrained_path

                self.trainers.append(current_trainer)

        return self

    def make_step(self, step, dataset):
        test_idx = index_slicer.create_idx_test(
            dataset.data,
            self.horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )
        target_idx = index_slicer.create_idx_target(
            dataset.data,
            self.horizon,
            self.history,
            self.step,
            date_column=dataset.date_column,
            delta=dataset.delta,
        )[:, self.model_horizon * step : self.model_horizon * (step + 1)]

        data = self.pipeline.create_data_dict_for_pipeline(dataset, test_idx, target_idx)
        data = self.pipeline.transform(data)

        pred = self.trainers[step].predict(data, self.pipeline)
        pred = self.pipeline.inverse_transform_y(pred)

        dataset.data.loc[target_idx.reshape(-1), dataset.target_column] = pred.reshape(-1)

        return dataset

class FlatWideMIMOStrategy(MIMOStrategy):
    def __init__(
        self,
        horizon: int,
        history: int,
        trainer: Union[MLTrainer, DLTrainer],
        pipeline: Pipeline,
        step: int = 1,
    ):
        super().__init__(horizon, history, trainer, pipeline, step)
        self.strategy_name = "FlatWideMIMOStrategy"

# Loading Data

To make it easier to track and understand how features and targets are generated depending on the strategy, we will use a synthetic series with the following structure:
- 10 series, each with 1000 points.
- The value in the `value` column is represented as `{id - 1}{point}`, meaning it is a concatenation of the series ID and the time point number within that series. For example, the value `5234` corresponds to the 234th point in the series with `id=4`.

In [13]:
df = pd.read_csv("./datasets/global/simulated_data_to_check.csv")
display(df.iloc[:10])
display(df.iloc[2000:2010])

,value,date,id
0,1000,2020-01-01,0
1,1001,2020-01-02,0
2,1002,2020-01-03,0
3,1003,2020-01-04,0
4,1004,2020-01-05,0
5,1005,2020-01-06,0
6,1006,2020-01-07,0
7,1007,2020-01-08,0
8,1008,2020-01-09,0
9,1009,2020-01-10,0


,value,date,id
2000,3000,2020-01-01,2
2001,3001,2020-01-02,2
2002,3002,2020-01-03,2
2003,3003,2020-01-04,2
2004,3004,2020-01-05,2
2005,3005,2020-01-06,2
2006,3006,2020-01-07,2
2007,3007,2020-01-08,2
2008,3008,2020-01-09,2
2009,3009,2020-01-10,2


In [14]:
horizon = 3
history = 7

In [15]:
dataset_params = {
    "target": {
        "columns": ["value"],
        "type": "continious",
    },
    "date": {
        "columns": ["date"],
        "type": "datetime",
    },
    "id": {
        "columns": ["id"],
        "type": "categorical",
    }
}

dataset = TSDataset(
    data=df,
    columns_params=dataset_params,
    print_freq_period_info=True,
)

freq: Day; period: 1


In [16]:
pipeline_easy_params = {
    "target_lags": 7,
    "date_lags": 3,
}

# Configure the model parameters
model = CatBoost
model_params = {
    "loss_function": "MultiRMSE",
    "early_stopping_rounds": 100,
    "verbose": 500,
}

# Configure the validation parameters
validation = KFoldCrossValidator
validation_params = {
    "n_splits": 2,
}

trainer_params = {}
trainer = MLTrainer(
    model,
    model_params,
    validation,
    validation_params,
)

# Description of Strategies

__Multi-series prediction strategies:__

- **Local-modelling**:
  - An individual model for each time series. 
  - Each time series as independent from others.

- **Global-modelling**:
  - A single model for all time series.
  - Features created from each series do not overlap with other series. Series are related but modeled separately.

- **Multivariate-modelling**:
  - A single model for all time series. 
  - Features created from each series are concatenated at each time step. Try to capture dependencies between the series at the same time point.

**Multi-point-ahead prediction strategies:**

- **Recursive**:
	- One model is used for the entire forecast horizon. 
	- training: The model is trained to predict one point ahead.
	- prediction: The model iteratively predicts each point, using previous predictions to update the features in the test data.
	- Note 1: There is an option to use a “reduced” version, where features are generated for all test observations at once, and unavailable values are filled with NaN.
	- Note 2: Recursive can also be combined with the MIMO strategy, allowing the model to predict model_horizon points ahead at each step.

- **Direct**:
	- Individual models are trained for each point in the forecast horizon.
	- Note 1: There is an option to use "equal_train_size" option, where all models can be trained on the same X_train set, formed for the last model predicting h point. Only the target variable (y) is updated for each model, reducing the time spent generating new training sets.
	- Note 2: Direct can also be combined with MIMO, where each individual model predicts model_horizon points ahead.

- **MIMO (Multi-input-multi-output)**:
 	- One model is trained and used for the entire forecast horizon at once. 
	- Note 1: This strategy can also accommodate exogenous features (for local- or global-modelling strategies).

- **FlatWideMIMO**:
	- A hybrid of Direct and MIMO. One model is trained, but Direct’s features are deployed across the forecast horizon.
	- Note 1: To use FlatWideMIMO with date-related features, h lags of them must be included (with help of LagTransformer).

## MIMO — global

Let's take the `MIMO-global` strategy as the base strategy, and from there, we will explore all the others.

In [8]:
pipeline = Pipeline.easy_setup(dataset_params, pipeline_easy_params, multivariate=False)

strategy = MIMOStrategy(
    horizon=horizon,
    history=history,
    pipeline=pipeline,
    trainer=trainer,
)